# 一、优化模型参数
- 现在我们有了模型和数据，是时候通过优化数据上的参数来训练、验证和测试我们的模型了。
- 训练模型是一个迭代过程；在每次迭代中，模型都会对输出进行猜测，计算其猜测的误差（损失），收集误差相对于其参数的导数（如我们在上一节中看到的），并使用梯度下降优化这些参数。
- 有关此过程的更详细演练，请观看 3Blue1Brown 的有关反向传播的视频。

## 1、准备工作
- 我们加载前面有关数据集和数据加载器以及构建模型部分的代码。

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

## 2、超参数
- 超参数是可调整的参数，可让您控制模型优化过程。
- 不同的超参数值会影响模型训练和收敛速度（阅读有关超参数调整的更多信息）
- Number of Epochs - 迭代数据集的次数
- Batch Size - 参数更新之前通过网络传播的数据样本数量
- 学习率 - 每个批次/时期更新模型参数的量。较小的值会导致学习速度较慢，而较大的值可能会导致训练期间出现不可预测的行为。

In [2]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 3、优化循环
- 一旦我们设置了超参数，我们就可以使用优化循环来训练和优化我们的模型。优化循环的每次迭代称为一个epoch。
- 每个epoch由两个主要部分组成：
- 训练循环 - 迭代训练数据集并尝试收敛到最佳参数。
- 验证/测试循环 - 迭代测试数据集以检查模型性能是否有所改善。
- 让我们简单熟悉一下训练循环中使用的一些概念。向前跳转查看优化循环的完整实现。
- 损失函数：当提供一些训练数据时，我们未经训练的网络可能不会给出正确的答案。
- 损失函数衡量的是得到的结果与目标值的不相似程度，它是我们在训练时想要最小化的损失函数。
- 为了计算损失，我们使用给定数据样本的输入进行预测，并将其与真实数据标签值进行比较。
- 常见的损失函数包括用于回归任务的 nn.MSELoss（均方误差）和用于分类的 nn.NLLLoss（负对数似然）。
- nn.CrossEntropyLoss 结合了 nn.LogSoftmax 和 nn.NLLLoss。
- 我们将模型的输出 logits 传递给 nn.CrossEntropyLoss，这将标准化 logits 并计算预测误差。

In [3]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

## 4、优化器
- 优化是调整模型参数以减少每个训练步骤中模型误差的过程。
- 优化算法定义了如何执行此过程（在本例中我们使用随机梯度下降）。
- 所有优化逻辑都封装在优化器对象中。
- 这里，我们使用SGD优化器；此外，PyTorch 中还有许多不同的优化器，例如 ADAM 和 RMSProp，它们可以更好地处理不同类型的模型和数据。
- 我们通过注册需要训练的模型参数并传入学习率超参数来初始化优化器。

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
# 在训练循环中，优化分三个步骤进行：
# 调用optimizer.zero_grad()重置模型参数的梯度。默认情况下渐变相加；为了防止重复计算，我们在每次迭代时明确地将它们归零。
# 通过调用 loss.backward() 反向传播预测损失。 PyTorch 存储损失的梯度。每个参数。
# 一旦我们得到了梯度，我们就调用optimizer.step()来通过向后传递中收集的梯度来调整参数。

In [6]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    # 获取模型
    model.train()
    # 加载数据 batch 批次数据 X：输入数据, y：真实输出数据
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        # optimizer.step()则利用这些计算出的梯度，根据所选择的优化算法（如SGD，Adam等）和设定的学习率，来更新模型的参数，从而最小化损失函数。
        optimizer.step()
        # 清空（或重置）优化器中累积的梯度
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * batch_size + len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    # 模型设置为评估（evaluation）模式
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    # 不用反向传播计算梯度
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    # 计算准确度
    correct /= size
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.315257  [   64/60000]
loss: 2.293368  [ 6464/60000]
loss: 2.278170  [12864/60000]
loss: 2.262481  [19264/60000]
loss: 2.244125  [25664/60000]
loss: 2.221610  [32064/60000]
loss: 2.223866  [38464/60000]
loss: 2.186689  [44864/60000]
loss: 2.194059  [51264/60000]
loss: 2.149075  [57664/60000]
Test Error: 
 Accuracy: 40.6%, Avg loss: 2.151281 

Epoch 2
-------------------------------
loss: 2.173686  [   64/60000]
loss: 2.148438  [ 6464/60000]
loss: 2.096913  [12864/60000]
loss: 2.104583  [19264/60000]
loss: 2.054512  [25664/60000]
loss: 2.004362  [32064/60000]
loss: 2.023711  [38464/60000]
loss: 1.937719  [44864/60000]
loss: 1.950864  [51264/60000]
loss: 1.870941  [57664/60000]
Test Error: 
 Accuracy: 56.5%, Avg loss: 1.879006 

Epoch 3
-------------------------------
loss: 1.921463  [   64/60000]
loss: 1.872409  [ 6464/60000]
loss: 1.769786  [12864/60000]
loss: 1.798491  [19264/60000]
loss: 1.689537  [25664/60000]
loss: 1.660327  [32064/600